#### 📦 Install & Import Libraries

In [ ]:
# Install missing packages if needed (usually optional in Colab)
!pip install emoji praw spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 10.9 MB/s eta 0:00:00


In [ ]:
# Import necessary Python libraries
import praw
import pandas as pd
import re
import nltk
import spacy
import emoji
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from google.colab import files

In [ ]:
# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

#### 🔑 Connect to Reddit API

In [ ]:
# Download spaCy English model
spacy.cli.download("en_core_web_sm")

# Setup Reddit API credentials (replace with yours if needed)
reddit = praw.Reddit(
    client_id='6W3smle2IjF6q1LM7PcAFg',
    client_secret='csRmN3HWdV9DoTShBgqbjqwQ1QOmAA',
    user_agent='sentiment-analysis-malaysia'
)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


####📥 Scrape Reddit Posts Comments from r/malaysia

In [ ]:
from IPython.display import display, HTML
from datetime import datetime


# Collect latest 1000 'hot' posts from r/malaysia
subreddit = reddit.subreddit('malaysia')
posts = []
for post in subreddit.hot(limit=1000):
    posts.append([
        post.title, #The title of the post
        post.selftext, #The text body of the post
        post.score, #Number of upvotes minus downvotes
        post.num_comments, #Number of comments on the post
        post.created_utc, #Timestamp when post was created (in UTC)
        post.id, #Unique Reddit post ID
        post.url #Link to the Reddit post
    ])

# Create DataFrame from Reddit data
df = pd.DataFrame(posts, columns=['Title', 'Body', 'Score', 'Number Of Comments', 'Timestamp', 'Post Id', 'Url'])

#Combine Title and Body into a single column
df['Full Text'] = df['Title'] + ' ' + df['Body']

# Format timestamp
df['Timestamp'] = df['Timestamp'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

# Make URL clickable
df['Url'] = df['Url'].apply(lambda x: f'<a href="{x}" target="_blank">{x}</a>')

df.to_csv('r_malaysia_raw.csv', index=False)

# Show sample of raw dataset
print("📄 First 5 rows of RAW Reddit data:")
display(HTML(df[['Title', 'Body', 'Score', 'Number Of Comments', 'Timestamp', 'Post Id', 'Url']].head(5).to_html(escape=False)))


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

📄 First 5 rows of RAW Reddit data:


,Title,Body,Score,Number Of Comments,Timestamp,Post Id,Url
0,/r/Malaysia daily random discussion and quick questions thread for 18 June 2025,"This is [r/malaysia](https://www.reddit.com/r/malaysia/)'s official daily random discussion and quick questions thread. Don't be shy! Share your joys, frustrations, random thoughts and questions. Anything and everything is welcome.\n\n>*Jom tengok DT pada awal pagi* \n> \n>*Semoga semua monyet sihat* \n> \n>*Nasi apa yang orang suka bagi?* \n> \n>*Sudah semestinya bagi nasihat*",1,31,2025-06-17 20:00:37,1ldx8x4,https://www.reddit.com/r/malaysia/comments/1ldx8x4/rmalaysia_daily_random_discussion_and_quick/
1,Cat taking the escalator down.. probably the most Malaysian thing I've seen today. Please watch you tail!,,766,43,2025-06-18 01:44:43,1le513c,https://v.redd.it/9t7kssn4cl7f1
2,Pervert get caught at ipoh mall,"The girl victim said on her Facebook page: \n\nOn June 15, my sister was shopping in a mall in Ipoh. She suddenly turned around and found that this pervert took his phone and quickly took it out from under my sister's skirt. Her boyfriend stepped forward to question him, but the pervert denied it. Her boyfriend immediately snatched the pervert's phone and asked him for the password. Then he found that the album was full of other girls' voyeuristic videos, all of which were underskirt videos and breast close-ups!!\n\nImmediately told the clerk for help. I am very grateful to the female clerk for helping to call the police after understanding the situation. The clerk also said that this person often wandered around their store. They just thought he wanted to steal things. I hope that girls will pay more attention outside in the future. If you find similar incidents, you must bravely stop and call the police. Don't let these perverts get away with it!",207,54,2025-06-18 02:12:31,1le5kue,https://v.redd.it/basksuqggl7f1
3,Nga Kor Ming brought MPs on a tour to demonstrate the importance of the Urban Renewal Act,,157,85,2025-06-17 23:42:05,1le2iqr,https://v.redd.it/wvzv28u8qk7f1
4,"Alleged Serial Upskirt Photographer Caught By Students In Bukit Bintang, ""500GB Of Videos Found""",,881,136,2025-06-17 14:03:52,1ldnz48,https://v.redd.it/y6gk79czuh7f1


In [ ]:
from IPython.display import display, HTML
from datetime import datetime
import time # Import the time module

print("\n--- Scraping Comments ---")

all_comments_data = []

# Iterate through each post in the DataFrame to get its comments
for index, row in df.iterrows():
    post_id = row['Post Id']
    submission = reddit.submission(id=post_id)
    submission.comments.replace_more(limit=None) # This expands all "More Comments" links

    for comment in submission.comments.list():
        all_comments_data.append({
            'Post ID': post_id,
            'Comment ID': comment.id,
            'Comment Body': comment.body,
            'Comment Score': comment.score,
            'Comment Timestamp': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
            'Comment Author': comment.author.name if comment.author else '[deleted]',
            'Comment URL': f"https://www.reddit.com{comment.permalink}"
        })
    time.sleep(5) # Increased delay to 5 seconds


# Create a DataFrame for comments
df_comments = pd.DataFrame(all_comments_data)

# Make Comment URL clickable
df_comments['Comment URL'] = df_comments['Comment URL'].apply(lambda x: f'<a href="{x}" target="_blank">{x}</a>')

df_comments.to_csv('r_malaysia_comments.csv', index=False)

print("\n📄 First 5 rows of scraped comments:")
display(HTML(df_comments.head(5).to_html(escape=False)))

print(f"\nSuccessfully scraped {len(all_comments_data)} comments and saved to 'r_malaysia_comments.csv'")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




--- Scraping Comments ---


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l


📄 First 5 rows of scraped comments:


,Post ID,Comment ID,Comment Body,Comment Score,Comment Timestamp,Comment Author,Comment URL
0,1ldx8x4,mybo1l1,"\n**Minor announcements:**\n\n* [2025 r/Malaysia Census](https://www.reddit.com/r/malaysia/comments/1jk23po/welcome_to_the_rmalaysia_census_2025/): Collecting Responses until 11.59pm, 23rd April 2025.\n* [Mental health wiki](https://www.reddit.com/r/malaysia/wiki/mental_health/): A list of mental health services in Malaysia\n \n\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/malaysia) if you have any questions or concerns.*",1,2025-06-17 20:00:37,AutoModerator,https://www.reddit.com/r/malaysia/comments/1ldx8x4/rmalaysia_daily_random_discussion_and_quick/mybo1l1/
1,1ldx8x4,mydlsxd,https://preview.redd.it/sj033twngl7f1.jpeg?width=2048&format=pjpg&auto=webp&s=cc86336fbd235d3b832792c56f89e8422c546456\n\nGot this from someone not long ago. THIS SHIT GOODDDDD.,1,2025-06-18 02:12:19,LongjumpingAnteater6,https://www.reddit.com/r/malaysia/comments/1ldx8x4/rmalaysia_daily_random_discussion_and_quick/mydlsxd/
2,1ldx8x4,mye9jjc,"I'm baffled by some buyers in Carousell. setting aside those who don't read the details of the listing, there are those who ask for ""my best price"". obviously my ""best price"" is the price where I can get the most amount, which is usually the price I listed. shouldn't they be the ones presenting me a price they want and see if I agree with the offer?",1,2025-06-18 04:48:05,aryehgizbar,https://www.reddit.com/r/malaysia/comments/1ldx8x4/rmalaysia_daily_random_discussion_and_quick/mye9jjc/
3,1ldx8x4,myby13z,"Hello, this is /u/ChubbyTrain's goth girlfriend speaking. My name is Aurora Crimson Velvet Nightwhisper. You all haven't seen me because I go to another school. \n\n[/u/ChubbyTrain took a picture of me the other day.](https://imgur.com/a/oiQJc6D) Ignore my extra limb, extra fingers, or my lack of neck, that's just /u/ChubbyTrain's camera glitching. \n\nDon't be mean to /u/ChubbyTrain, okay? I know karate. 😤 \n\nBe nice, or I and other 1000 bajillion REAL friends of /u/ChubbyTrain will karate all of you. We are REAL, okay? REAL, in capital letters! We're totally not AI imaginary friends. \n\n\n(Ini pos tahi. ✉️💩)",2,2025-06-17 20:47:23,ChubbyTrain,https://www.reddit.com/r/malaysia/comments/1ldx8x4/rmalaysia_daily_random_discussion_and_quick/myby13z/
4,1ldx8x4,mycgll9,How does it feel like to have Covid again in 2025?,1,2025-06-17 22:20:48,BrandonTeoh,https://www.reddit.com/r/malaysia/comments/1ldx8x4/rmalaysia_daily_random_discussion_and_quick/mycgll9/



Successfully scraped 32054 comments and saved to 'r_malaysia_comments.csv'


#### 🧼 Clean and Preprocess Text using NLP

In [ ]:
# Load spaCy model and other NLP tools
nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Define function to clean text
def clean_text(text):
    if not isinstance(text, str): # Ensure text is a string
        return ""
    text = text.lower()  # Lowercase all words
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = emoji.replace_emoji(text, replace='')  # Remove emojis
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation and numbers
    tokens = nltk.word_tokenize(text)  # Tokenize the text
    # Remove stopwords, short words, and apply lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 2]
    return ' '.join(tokens)  # Return cleaned text as a string

# Apply the cleaning function to the 'Comment Body' column of df_comments
df_comments['Cleaned_text'] = df_comments['Comment Body'].apply(clean_text)

# Save cleaned comment data
df_cleaned_comments = df_comments[['Cleaned_text', 'Comment Score', 'Comment Timestamp', 'Post ID', 'Comment ID']]
df_cleaned_comments.to_csv('cleaned_r_malaysia_comments.csv', index=False)

# Show sample of cleaned comment data
print("\n🧹 First 5 rows of CLEANED comment data:")
display(df_cleaned_comments.head())


🧹 First 5 rows of CLEANED comment data:


,Cleaned_text,Comment Score,Comment Timestamp,Post ID,Comment ID
0,minor announcement rmalaysia census collecting...,1,2025-06-17 20:00:37,1ldx8x4,mybo1l1
1,got someone long ago shit gooddddd,1,2025-06-18 02:12:19,1ldx8x4,mydlsxd
2,baffled buyer carousell setting aside dont rea...,1,2025-06-18 04:48:05,1ldx8x4,mye9jjc
3,hello uchubbytrains goth girlfriend speaking n...,2,2025-06-17 20:47:23,1ldx8x4,myby13z
4,feel like covid,1,2025-06-17 22:20:48,1ldx8x4,mycgll9


####📤 Download CSVs to Your Local Machine

In [ ]:
# Download both raw and cleaned CSV files to your computer
files.download('r_malaysia_comments.csv')
files.download('cleaned_r_malaysia_comments.csv')

print("✅ Download complete: 'r_malaysia_comments_raw.csv' and 'cleaned_r_malaysia_comments.csv'")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download complete: 'r_malaysia_comments_raw.csv' and 'cleaned_r_malaysia_comments.csv'
